# Identifying Fraudulent Activities
__- Author: Fan Yuan__  
__- Date: 03/27/2019__

## Context:
Employee turn-over is a very costly problem for companies. It's important to understand why and when employees are most likely to leave can lead to actions to improve employee retention as well as planning new hiring in advance.

## Project goal:
* Given the information about the employees, predict when employees are going to quit by understanding the main drivers of employee churn  
* Find the main factors that drive employee churn. Explain
* What variable can be added to the data set to help explain the employee churn


___NOTE:
Assume, for each company, that the headcount starts form 0 on 2011/01/23. Estimate employee headcount, for each company, on each day, from 2011/01/24 to 2015/12/13. That is, if by 2012/03/02 2000 people have joined company A and 1000 of them have already quit, then company headcount on 2012/03/02 would be 1000___


## Data:
* The data is from employees who joined from 2011/01/24 to 2015/12/13. For each employee, we also know if they are still at the company as of 2015/12/13 or they have quit. Beside that, we have general information about the employee, such as average salary during her tenure, dept, and years of experience


### employee_retention -- Columns:
* __employee_id__: id of the employee. Unique by employee per company
* __company_id__: company id
* __dept__: employee dept
* __seniority__: number of years of work experience when hired
* __salary__: average yearly salary of the employee during her tenure within the company
* __join_date__: when the employee joined the company. it can only be between 2011/01/24 and 2015/12/13
* __quit_date__: when the employee left her jon (if she is still employed as of 2015/12/13, this field is NA)
